In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("GCSExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#  Google Storage File Path
gsc_file_path = 'gs://data_de2022/sales.csv'  #  use your gcp bucket name. Also upload sales.csv first
# Create data frame
df = spark.read.format("csv").option("header", "true") \
       .load(gsc_file_path)
df.printSchema()

newDf = df.select('unit_sales')  # select one column

newDf.show()

df.show()

root
 |-- product_id: string (nullable = true)
 |-- time_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- promotion_id: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- store_sales: string (nullable = true)
 |-- store_cost: string (nullable = true)
 |-- unit_sales: string (nullable = true)

+----------+
|unit_sales|
+----------+
|         2|
|         3|
|         1|
|         3|
|         2|
|         3|
|         2|
|         2|
|         2|
|         1|
|         2|
|         2|
|         2|
|         1|
|         1|
|         2|
|         2|
|         1|
|         3|
|         1|
+----------+
only showing top 20 rows

+----------+-------+-----------+------------+--------+-----------+----------+----------+
|product_id|time_id|customer_id|promotion_id|store_id|store_sales|store_cost|unit_sales|
+----------+-------+-----------+------------+--------+-----------+----------+----------+
|       337|    371|       6280|        null|       2|

how many distinct products have been sold in each store? 


In [2]:
dfw = df.select("product_id","store_id").distinct()
dfw.show(2)

+----------+--------+
|product_id|store_id|
+----------+--------+
|       955|       2|
|      1119|       2|
+----------+--------+
only showing top 2 rows



In [3]:
dfw.write.format("csv").save("gs://data_de2022/lab65out.csv") # use correct bucket name

In [4]:
# Stop the spark context
spark.stop()